In [0]:
# from sklearn.neighbors import NearestNeighbors # KNN Clustering 
# from scipy.sparse import csr_matrix # Compressed Sparse Row matrix
# from sklearn.decomposition import TruncatedSVD # Dimensional Reduction


In [0]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

In [0]:
data = pd.read_csv("femdaily.csv")
data = data.dropna()
data.drop_duplicates(keep = "first", inplace = True)

In [0]:
data.drop(['Recommend','Date','Date Scrape','Url','Category'], axis=1, inplace=True)
data.rename(columns={'Reviewer':'reviewer','Product':'product','Stars':'rating'}, inplace=True)

In [4]:
mean_ = data.groupby(by="reviewer",as_index=False)['rating'].mean()
rating_avg = pd.merge(data,mean_,on='reviewer')
rating_avg['avg_rating']=rating_avg['rating_x']-rating_avg['rating_y']
rating_avg.head()

,reviewer,rating_x,product,rating_y,avg_rating
0,Ayuika,3,Perfect 3D Gel,3.090909,-0.090909
1,Ayuika,2,Instabright Tone Up Cream,3.090909,-1.090909
2,Ayuika,4,Gokujyun Ultimate Moisturizing Milk,3.090909,0.909091
3,Ayuika,3,Aloe Vera 92% Soothing Gel,3.090909,-0.090909
4,Ayuika,4,Acne Spot Treatment,3.090909,0.909091


In [5]:
final = pd.pivot_table(rating_avg,values='avg_rating',index='reviewer',columns='product')
final.head()

product,"0,2 mm Therapy Air Mask Sheet",10% Lactic Acid (Infused with Rose),100% Cold-Pressed Virgin Marula Oil,100% Organic Cold-Pressed Borage Seed Oil,100% Organic Cold-Pressed Rose Hip Seed Oil,100% Plant-Derived Squalane,100% Pure Camellia Dry Oil,100% Snail mucus Soothing Gel,14% AHA Swipes,2 Minutes Brightening Gel,24K Bio Gold Night Cream,24K Bio-Gold Rose Gold Water,24K Gold Premium First Essence,24K Gold Pure Luxury Cleansing Butter,24k Bio Gold Gold Water,25% mandelic acid peel,3 Step Nose Clear Pad,30% Mandelic Acid Anti-Acne Gel,7 Day Scrub Cream,7 Days Mask,8% Alpha Hydroxy Acid Gel,9 to 5,95% Watermelon Soothing Gel,98% Watermelon Soothing Gel,99% ALOE SOOTHING GEL,99% Aloe Vera Gelly,99% Jeju Fresh Aloe Soothing Gel,99% Snail and Gold Soothing Gel,@haple.id Almond Oil and Grapeseed Oil,A Perfect World Deep Cleanser,A'BLOOM Anti Blemish Peach Mask,A'BLOOM Brightening Lemon Lime Mask,A'BLOOM Moisturizing Watermelon Mask,A'PIEU Madecassoside Cream,A:PE Proatin Mask,AC Clean Up Gel Lotion,AC Clean Up Pink Powder Mask,AC Clean Up Toner,AC Clean up,AC Clinic Daily Gel Lotion,...,off with those heads,olive real oil mist EX,orange sleeping mask,oxy10 maximum acne pimple medication,perfect hydrating treatment esence,pond's glitter cream,pulchra.id,pure balance hydro gel moisturizer,pure source pocket pack,qiansoto peel off masque,real nature mask sheet,runaskin peel off,serum envygreen,shoppasoap cleansing balm,silcot maximizer cotton,skin &amp; lab gently vita exfoliator,snail shooting gel the saem,some by mi,thayers alcohol free toner,the body shop tea tree serum,the littles kit,the polished,the water cream,ultimate moisturising honey,uv mild milk,viva milk cleanser dan viva face tonic,viva skinfood,wardah mineral clay mask,wardah nature daily,wardah seaweed balancing facial wash,wardah seaweed intensive night cream,wardah suncare,watermelon glow,white beauty aqua facial wash with plant extract,whitening plus complex,whitening soap,wonder pore,zaitun oil,“Buffet”,“B” oil
reviewer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01lely,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01putrisalma,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
01sary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123hayoapa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

In [7]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
final_user

product,"0,2 mm Therapy Air Mask Sheet",10% Lactic Acid (Infused with Rose),100% Cold-Pressed Virgin Marula Oil,100% Organic Cold-Pressed Borage Seed Oil,100% Organic Cold-Pressed Rose Hip Seed Oil,100% Plant-Derived Squalane,100% Pure Camellia Dry Oil,100% Snail mucus Soothing Gel,14% AHA Swipes,2 Minutes Brightening Gel,24K Bio Gold Night Cream,24K Bio-Gold Rose Gold Water,24K Gold Premium First Essence,24K Gold Pure Luxury Cleansing Butter,24k Bio Gold Gold Water,25% mandelic acid peel,3 Step Nose Clear Pad,30% Mandelic Acid Anti-Acne Gel,7 Day Scrub Cream,7 Days Mask,8% Alpha Hydroxy Acid Gel,9 to 5,95% Watermelon Soothing Gel,98% Watermelon Soothing Gel,99% ALOE SOOTHING GEL,99% Aloe Vera Gelly,99% Jeju Fresh Aloe Soothing Gel,99% Snail and Gold Soothing Gel,@haple.id Almond Oil and Grapeseed Oil,A Perfect World Deep Cleanser,A'BLOOM Anti Blemish Peach Mask,A'BLOOM Brightening Lemon Lime Mask,A'BLOOM Moisturizing Watermelon Mask,A'PIEU Madecassoside Cream,A:PE Proatin Mask,AC Clean Up Gel Lotion,AC Clean Up Pink Powder Mask,AC Clean Up Toner,AC Clean up,AC Clinic Daily Gel Lotion,...,off with those heads,olive real oil mist EX,orange sleeping mask,oxy10 maximum acne pimple medication,perfect hydrating treatment esence,pond's glitter cream,pulchra.id,pure balance hydro gel moisturizer,pure source pocket pack,qiansoto peel off masque,real nature mask sheet,runaskin peel off,serum envygreen,shoppasoap cleansing balm,silcot maximizer cotton,skin &amp; lab gently vita exfoliator,snail shooting gel the saem,some by mi,thayers alcohol free toner,the body shop tea tree serum,the littles kit,the polished,the water cream,ultimate moisturising honey,uv mild milk,viva milk cleanser dan viva face tonic,viva skinfood,wardah mineral clay mask,wardah nature daily,wardah seaweed balancing facial wash,wardah seaweed intensive night cream,wardah suncare,watermelon glow,white beauty aqua facial wash with plant extract,whitening plus complex,whitening soap,wonder pore,zaitun oil,“Buffet”,“B” oil
reviewer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
01lely,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
01putrisalma,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
01sary,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0

In [8]:
b = cosine_similarity(final_user)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
similarity_with_user.head()

reviewer,,01lely,01putrisalma,01sary,123hayoapa,15ayusafitri,15popipu,1602pika,1994pj,1Unni,1m_human,2000,21ChocLatte,2309anisa,2712dita,28zura,3rikganteng,727wuehdsh,777Cell,89rina,94echon,98rahmah,990lush,99shinta,9iserin,@dday12,@ulfahh,AARIANDAYU,ADLusari,AL1ce,ANGGUNNUGRAHA,APRILYANIA,ARK7,AYUKSM,A_nicky,Aanditaa,Abbyfreya,AbdelFikar,Abelcantika,Abigaille,...,zizaaulia,zizahay,zizahnrh,ziziput,zkhs1709,zkshafira,zlindra,zmihrt,zoee,zoellva,zoey_stellar,zoeydya,zoeyzoe,zofiazora,zonaisyah,zoomintz,zrfh30,zrna,zsabrinna,zsalsabil,zsalzsamaulida,zshintyaariza,ztdnamn,zufirai,zuhrotulmufidah,zulaikaheffendi,zulfadewi,zulfiakamila,zulfiapf,zulfidah,zulfyahanny,zuliettaz,zulvveh,zuuuuu,zuyinazui,zvnazole,zyshalu,zzarahs,zzfatimah,zzulia
reviewer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,0.000,0.000,0.000,0.337,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.111,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000
01lely,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.102,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.137,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.014,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
01putrisalma,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
01sary,0.337,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
123hayoapa,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,...,-0.103,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,0.012,-0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-0.000,-0.000,0.000,0.000,-0.000,0.000


In [0]:
# Replacing NaN by product Average
final_prod = final.fillna(final.mean(axis=0))

In [10]:
# user similarity on replacing NAN by item avg
cosine = cosine_similarity(final_prod)
np.fill_diagonal(cosine, 0 )
similarity_with_product = pd.DataFrame(cosine,index=final_prod.index)
similarity_with_product.columns=final_user.index
similarity_with_product.head()

reviewer,,01lely,01putrisalma,01sary,123hayoapa,15ayusafitri,15popipu,1602pika,1994pj,1Unni,1m_human,2000,21ChocLatte,2309anisa,2712dita,28zura,3rikganteng,727wuehdsh,777Cell,89rina,94echon,98rahmah,990lush,99shinta,9iserin,@dday12,@ulfahh,AARIANDAYU,ADLusari,AL1ce,ANGGUNNUGRAHA,APRILYANIA,ARK7,AYUKSM,A_nicky,Aanditaa,Abbyfreya,AbdelFikar,Abelcantika,Abigaille,...,zizaaulia,zizahay,zizahnrh,ziziput,zkhs1709,zkshafira,zlindra,zmihrt,zoee,zoellva,zoey_stellar,zoeydya,zoeyzoe,zofiazora,zonaisyah,zoomintz,zrfh30,zrna,zsabrinna,zsalsabil,zsalzsamaulida,zshintyaariza,ztdnamn,zufirai,zuhrotulmufidah,zulaikaheffendi,zulfadewi,zulfiakamila,zulfiapf,zulfidah,zulfyahanny,zuliettaz,zulvveh,zuuuuu,zuyinazui,zvnazole,zyshalu,zzarahs,zzfatimah,zzulia
reviewer,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
,0.000,0.983,0.992,0.993,0.990,0.987,0.980,0.989,0.991,0.991,0.991,0.992,0.992,0.991,0.990,0.976,0.992,0.993,0.992,0.993,0.992,0.992,0.992,0.991,0.993,0.988,0.992,0.993,0.987,0.993,0.992,0.988,0.992,0.992,0.989,0.992,0.992,0.992,0.993,0.992,...,0.969,0.992,0.992,0.991,0.993,0.991,0.993,0.990,0.993,0.992,0.991,0.993,0.992,0.991,0.981,0.992,0.990,0.987,0.990,0.992,0.993,0.992,0.992,0.992,0.989,0.991,0.992,0.991,0.992,0.990,0.991,0.985,0.992,0.992,0.991,0.992,0.993,0.993,0.992,0.991
01lely,0.983,0.000,0.990,0.989,0.988,0.985,0.978,0.987,0.989,0.989,0.989,0.990,0.990,0.989,0.988,0.974,0.990,0.991,0.990,0.991,0.991,0.990,0.990,0.989,0.991,0.986,0.990,0.991,0.985,0.991,0.990,0.986,0.990,0.991,0.987,0.990,0.990,0.990,0.991,0.990,...,0.967,0.990,0.990,0.989,0.991,0.990,0.991,0.988,0.991,0.990,0.990,0.991,0.989,0.989,0.979,0.990,0.988,0.985,0.988,0.990,0.991,0.990,0.990,0.990,0.987,0.989,0.990,0.989,0.990,0.988,0.989,0.983,0.990,0.991,0.989,0.990,0.991,0.991,0.990,0.989
01putrisalma,0.992,0.990,0.000,0.998,0.997,0.993,0.987,0.996,0.998,0.998,0.997,0.999,0.998,0.997,0.997,0.983,0.999,0.999,0.999,0.999,0.999,0.999,0.999,0.998,0.999,0.995,0.998,0.999,0.994,0.999,0.999,0.995,0.998,0.999,0.996,0.999,0.999,0.999,0.999,0.999,...,0.976,0.999,0.999,0.998,0.999,0.998,0.999,0.996,0.999,0.999,0.998,0.999,0.999,0.998,0.987,0.999,0.996,0.993,0.997,0.998,0.999,0.999,0.999,0.999,0.996,0.998,0.999,0.998,0.999,0.996,0.998,0.991,0.999,0.999,0.997,0.998,0.999,0.999,0.999,0.998
01sary,0.993,0.989,0.998,0.000,0.996,0.992,0.986,0.995,0.997,0.997,0.996,0.998,0.997,0.996,0.996,0.982,0.998,0.998,0.998,0.998,0.998,0.998,0.998,0.997,0.998,0.994,0.997,0.998,0.993,0.998,0.998,0.994,0.998,0.998,0.995,0.998,0.998,0.998,0.998,0.998,...,0.975,0.998,0.998,0.997,0.998,0.997,0.998,0.995,0.998,0.998,0.997,0.998,0.998,0.997,0.987,0.998,0.995,0.992,0.996,0.997,0.998,0.998,0.998,0.998,0.995,0.997,0.998,0.997,0.998,0.995,0.997,0.990,0.998,0.998,0.996,0.997,0.998,0.998,0.998,0.997
123hayoapa,0.990,0.988,0.997,0.996,0.000,0.992,0.985,0.994,0.996,0.996,0.996,0.997,0.997,0.996,0.995,0.981,0.997,0.998,0.997,0.998,0.998,0.998,0.998,0.996,0.998,0.993,0.997,0.998,0.992,0.998,0.997,0.993,0.997,0.998,0.994,0.998,0.997,0.997,0.998,0.997,...,0.972,0.997,0.998,0.996,0.998,0.997,0.998,0.995,0.998,0.997,0.997,0.998,0.997,0.996,0.986,0.997,0.995,0.992,0.995,0.997,0.998,0.997,0.997,0.997,0.994,0.996,0.997,0.996,0.998,0.995,0.996,0.990,0.997,0.997,0.996,0.997,0.998,0.998,0.997,0.996


In [0]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df

In [12]:
# top 10 neighbours for each user
sim_user_10_u = find_n_neighbours(similarity_with_user,10)
sim_user_10_u.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
reviewer,,,,,,,,,,
,EllenM,wafiyyaw,01sary,ulfihardianti,nella99_,prisillamonica,Ovaalpha,wildamaulida18,Hariyani,melon97
01lely,nastitidian,rhenaindria,dibidibidis,ameliasulimin,miz_purple,vicaar,yulinsar,MelatiSulistia,putuwidishintia,aintsalsha
01putrisalma,Ayudevv,wheewheawie,netanyarut,rositadwii,threed,aqilafyz,sherylesf,vindakhaz,monicakemuning,fathirzky
01sary,natasya_pw,Donnatri,nisamanda21,santydamayanti26,nella99_,prisillamonica,ulfihardianti,nishee,happykid,EllenM
123hayoapa,suciwisa,husnaghniya,Elysiab,dewiarinii,Mpiih,SarchillCerish,clarakaizer,juliezhao_jz,diahmeirina,nike256


In [13]:
# top 10 neighbours for each user
sim_user_10_m = find_n_neighbours(similarity_with_product,10)
sim_user_10_m.head()

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10
reviewer,,,,,,,,,,
,EllenM,shanazs,nella99_,vaolinda,prisillamonica,wildamaulida18,airinasysyahra,dianairmas,ulfihardianti,mpiyoo
01lely,zenitl,cides,rhenaindria,rossilinda,yunita670,miz_purple,aintsalsha,risyanthi,Iuliul,yeshasn
01putrisalma,wheewheawie,fathirzky,sitiamalya,athyaadhr,herdaps,shevinna,Trysiass,sarnisrina,Rahmasari7,rissamustikas
01sary,Donnatri,santydamayanti26,happykid,nishee,Shanisafitria,nimoku,nisamanda21,hanjaylien,Febryola,MutiaSuherman
123hayoapa,SarchillCerish,Elysiab,bdesvina,ranasabh,dewiarinii,nastityadw,NurElda,ivanaoktaa,Fiodora01,vaniannisa


In [0]:
def get_user_similar_product( user1, user2 ):
    common_product = rating_avg[rating_avg.reviewer == user1].merge(
    rating_avg[rating_avg.reviewer == user2],
    on = "product",
    how = "inner" )
    return common_product.merge( data, on = 'product' )

In [0]:
a = get_user_similar_product('01lely','yeshasn')
a = a.loc[ : , ['rating_x_x','rating_x_y','product']]


In [0]:
prod_rec = a.drop_duplicates(keep='first')

In [26]:
prod_rec

,rating_x_x,rating_x_y,product
0,3,4,Sparkling White Facial Foam
93,4,5,Milk Cleanser


In [0]:
rating_avg = rating_avg.astype({"product": str})
product_user = rating_avg.groupby(by = 'reviewer')['product'].apply(lambda x:','.join(x))

In [0]:
def User_item_score1(user):
    product_buy_by_user = final.columns[final[final.index==user].notna().any()].tolist()
    a = sim_user_10_m[sim_user_10_m.index==user].values
    b = a.squeeze().tolist()
    d = product_user[product_user.index.isin(b)]
    l = ','.join(d.values)
    product_buy_by_similar_users = l.split(',')
    product_under_consideration = list(set(product_buy_by_similar_users)-set(list(map(str, product_buy_by_user))))
    product_under_consideration = list(map(str, product_under_consideration))
    score = []
    for item in product_under_consideration:
        c = final_prod.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = mean_.loc[mean_['reviewer'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_product.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['avg_score','correlation']
        fin['score']=fin.apply(lambda x:x['avg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data_ = pd.DataFrame({'product':product_under_consideration,'score':score})
    top_5_recommendation = data_.sort_values(by='score',ascending=False).head(5)
    Product_Name = top_5_recommendation.merge(data, how='inner', on='product')
    Product_Names = Product_Name.product
    return Product_Names

In [59]:
user = str(input("Enter the user to whom you want to recommend : "))
predicted_product = User_item_score1(user)
#predicted_product_ = pd.DataFrame(data=predicted_product)
#predicted_product_.drop_duplicates(keep='first',inplace=True)
print(" ")
print("The Recommendations for User: 01lely")
print(predicted_product)

# for i in predicted_product:
#     print(i)

Enter the user id to whom you want to recommend : yeshasn
 
The Recommendations for User Id : 01lely
<bound method DataFrame.prod of           product  score          reviewer  rating
0      Lotion P50  5.175         chichago_       5
1      Lotion P50  5.175  intan_dwijayanti       4
2      Lotion P50  5.175         firameidi       5
3      Lotion P50  5.175      peacockgreen       5
4      Lotion P50  5.175       bee_hope289       5
..            ...    ...               ...     ...
579  Perfect Whip  4.757         AstriyaMC       5
580  Perfect Whip  4.757          aqilahmn       4
581  Perfect Whip  4.757         aisyafiya       5
582  Perfect Whip  4.757         chikavirg       5
583  Perfect Whip  4.757       arachmaulia       4

[584 rows x 4 columns]>
